In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.ticker as mtick

#Natural Language Toolkit/TextBlob
import re
import nltk as nlt
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

nlt.download('punkt')
nlt.download('stopwords')
nlt.download('wordnet')

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lighta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lighta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lighta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
rawtweets=pd.read_csv('tweets.csv',header=None,encoding='ISO-8859-1')

In [ ]:
df=df.append(df2, ignore_index=True)

EDA

In [3]:
rawtweets.head(10)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [4]:
rawtweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1600000 non-null  int64 
 1   1       1600000 non-null  int64 
 2   2       1600000 non-null  object
 3   3       1600000 non-null  object
 4   4       1600000 non-null  object
 5   5       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [4]:
tweets=rawtweets.copy()
#tweets=rawtweets.sample(20000,axis=0)

In [5]:
#Rename columns
old_col=[0,1,2,3,4,5]
new_col=['Label','ID','Date','Query','User','Text']
tweets[new_col]=tweets[old_col]
tweets=tweets.drop(old_col,axis=1)
tweets.head()

,Label,ID,Date,Query,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
#Format Date
tweets['Date']=pd.to_datetime(tweets['Date'])

C:\Users\Lighta\Anaconda3\envs\Explorer\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [8]:
tweets['Date'].dt.tz_localize('America/Los_Angeles')

0         2009-04-06 22:19:45-07:00
1         2009-04-06 22:19:49-07:00
2         2009-04-06 22:19:53-07:00
3         2009-04-06 22:19:57-07:00
4         2009-04-06 22:19:57-07:00
                     ...           
1599995   2009-06-16 08:40:49-07:00
1599996   2009-06-16 08:40:49-07:00
1599997   2009-06-16 08:40:49-07:00
1599998   2009-06-16 08:40:49-07:00
1599999   2009-06-16 08:40:50-07:00
Name: Date, Length: 1600000, dtype: datetime64[ns, America/Los_Angeles]

In [9]:
tweets.head()

,Label,ID,Date,Query,User,Text
0,0,1467810369,2009-04-06 22:19:45,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,2009-04-06 22:19:49,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,2009-04-06 22:19:53,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,2009-04-06 22:19:57,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,2009-04-06 22:19:57,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
#tweets=tweets.assign(Count=1)

In [11]:
def form_sentence (tweet):
    tweet_blob=TextBlob(tweet)
    return ''.join(tweet_blob.words)

In [12]:
print(form_sentence(tweets['Text'].iloc[12]))

caregivingIcouldn'tbeartowatchitAndIthoughttheUAlosswasembarrassing


In [13]:
print(tweets['Text'].iloc[12])

@caregiving I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . .


In [14]:
#Tokenizer (single string)
def no_user_alpha(tweet):
    tweet_list=[ele for ele in tweet.split() if ele !='user']
    clean_tokens=[t for t in tweet_list if
                re.match(r'[^\W\d]*$',t)]
    clean_s=''.join(clean_tokens)
    clean_mess=[word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
    return clean_tokens


In [15]:
print(no_user_alpha(tweets['Text'].iloc[12]))

['I', 'bear', 'to', 'watch', 'And', 'I', 'thought', 'the', 'UA', 'loss', 'was', 'embarrassing']


In [16]:
#Lemmatizer
def normalization(tweet_list):
    lem=WordNetLemmatizer()
    normalized_tweet=[]
    for word in tweet_list:
        normalized_text=lem.lemmatize(word,'v')
        normalized_tweet.append(normalized_text)
        return normalized_tweet

In [17]:
print(normalization(tweets['Text'].iloc[12].split()))

['@caregiving']


In [6]:
pipeline=Pipeline([
    ('bow',CountVectorizer(analyzer='word',stop_words='english')), #quantify text contents by word
    ('tfidf',TfidfTransformer()), #weighted TF-IDF scores
    ('classifier', RandomForestClassifier(n_estimators=50)), #train on TF-IDF vectors w SVC
])

In [7]:
X_train, X_test, y_train, y_test=train_test_split(tweets['Text'],tweets['Label'],test_size=0.2)

In [ ]:
pipeline.fit(X_train,y_train)

In [21]:
preds=pipeline.predict(X_test)

In [22]:
print(classification_report(preds,y_test))

              precision    recall  f1-score   support

           0       0.55      0.79      0.65    110756
           4       0.86      0.65      0.74    209244

    accuracy                           0.70    320000
   macro avg       0.70      0.72      0.69    320000
weighted avg       0.75      0.70      0.71    320000



In [ ]:
plt.figure()

# Add the models to the list that you want to view on the ROC plot
models = [
{
      'label': 'Support Vector Classifier',
      'model': SVC(probability=True)
},
{
    'label': 'Gradient Boosting',
    'model': GradientBoostingClassifier(n_estimators=50)
},
{
    'label': 'Random Forest',
    'model': RandomForestClassifier(n_estimators=50)
}
]
    
# Below for loop iterates through your models list
for m in models:
    model = m['model'] # select the model
    label = m['label'] # select the label
    
    pipeline=Pipeline([
    ('bow',CountVectorizer(analyzer='word',stop_words='english')), #quantify text contents by word
    ('tfidf',TfidfTransformer()), #weighted TF-IDF scores
    (label, model), #train on TF-IDF vectors w SVC
    ])
    
    pipeline.fit(X_train, y_train) # train the model
    y_pred=pipeline.predict(X_test)# predict the test data

    # Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pipeline.predict_proba(X_test)[:,1], pos_label=4)

    # Calculate Area under the curve to display on the plot
    auc = metrics.roc_auc_score(y_test,pipeline.predict(X_test))

    # Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))

    # Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.07])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity (False Positive Rate)')
plt.ylabel('Sensitivity (True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="best")
plt.show()   # Display